In [45]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [46]:
falencia_teste = pd.read_csv('falencia-teste.csv', header=0, delimiter=';')
falencia_treinamento = pd.read_csv('falencia-treinamento.csv', delimiter=';')
modelo_submissao = pd.read_csv('modelo-submissao.csv', header=0, delimiter=';')

In [47]:
#falencia_trinamento.head(10)

X = falencia_treinamento.values[:,0].reshape(-1,1) # Conjunto de atributos

y = falencia_treinamento.values[:,-1].reshape(-1,1) # Rotulo utilizado para regressao

In [48]:
modelo = LinearRegression()
modelo.fit(X, y)

LinearRegression()

In [50]:
y_pred = modelo.predict(X)

In [51]:
r_squared = r2_score(y, y_pred)

In [53]:
print("Coeficiente de determinação: %.4f" % modelo.score(X, y))

print("Intercept: %.4f" % modelo.intercept_)

# Slope
print('Slope: %.4f' % modelo.coef_)

Coeficiente de determinação: 0.0001
Intercept: 0.3008
Slope: 0.0000
